In [13]:
import pandas as pd
import os
import hashlib
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [14]:
file_path="~/SageMaker/Rain_Estimation_Using_Loosely_Matched_Stations/CMLs_20110609_20110813.dat"

In [15]:
df = pd.read_table(file_path, sep="\s+", index_col=False)

In [16]:
df["Start_Concat"]=df["YStart"].astype(str)+" "+df["XStart"].astype(str)
df["End_Concat"]=df["YEnd"].astype(str)+" "+df["XEnd"].astype(str)

In [17]:
def convert_to_four_letter_hash(concat):
    h = hashlib.sha1(str.encode(concat))
    d = h.digest()
    s = ""
    for i in range(0,4):
        x = d[i] % 52
        if x >= 26:
            s += chr(ord('A') + x - 26)
        else:
            s += chr(ord('a') + x)
    return s

In [18]:
print(f'{convert_to_four_letter_hash("51.80485 5.11687")}-{convert_to_four_letter_hash("51.83246 5.08996")}')


XmQw-fTxW


In [19]:
def convert_datetime_to_time(datetime):
    yyyy=datetime[:4]
    mm=datetime[4:6]
    dd=datetime[6:8]
    hh=datetime[-4:-2]
    ss=datetime[-2:]
    
    return(f"{dd}-{mm}-{yyyy} {hh}:{ss}")
    

In [20]:
df["ID"]=df["Start_Concat"].apply(convert_to_four_letter_hash).astype(str)+"-"+df["End_Concat"].apply(convert_to_four_letter_hash).astype(str)

In [21]:
df["Time"]=df["DateTime"].astype(str).apply(convert_datetime_to_time)

In [22]:
ids=df.ID.unique()

In [23]:
df["PowerTLTMmax[dBm]"]=5
df["PowerTLTMmin[dBm]"]=4.8
df.rename(columns={"Pmin": "PowerRLTMmin[dBm]", "Pmax": "PowerRLTMmax[dBm]"}, inplace=True)
df.dropna(inplace=True)

In [24]:
root="~/SageMaker/Rain_Estimation_Using_Loosely_Matched_Stations/CellEnMon/datasets/dme/09062011_13082011/raw"
columns=["Time","PowerTLTMmax[dBm]","PowerTLTMmin[dBm]","PowerRLTMmax[dBm]","PowerRLTMmin[dBm]"]
for id in ids[:100]:
    T,R=id.split("-")
    d=df[df.ID==id]
    d.to_csv(f'{root}/{T}_{d.YStart.iloc[0]}_{d.XStart.iloc[0]}_{R}_{d.YEnd.iloc[0]}_{d.XEnd.iloc[0]}.csv',index=False, columns=columns)
    

In [ ]:
df